<a href="https://colab.research.google.com/github/alebjanes/fire-susceptibility-mapping/blob/main/CNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import load
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation, InputLayer, BatchNormalization
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#DATA

pixel_size = 25
mid_pixel = pixel_size/2 - 0.5

Dataset = np.load('/content/drive/My Drive/MT/Samples/samples' + str(pixel_size) + 'x'+ str(pixel_size) +'_v3.npy')

X = Dataset[:,:,:,1:21]
target = Dataset[:,mid_pixel,mid_pixel,0]
y = np.expand_dims(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X=X_train
y=y_train

del y_test
del X_test
del target
del X_train
del y_train

In [ ]:
#CV
num_folds = 5

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []

acc_per_fold_train = []
loss_per_fold_train = []

In [ ]:
#Build CNN function (LULC paper)
def build_model(input_shape, batch_normalization, momentum, activation, loss, lr, optimizer):
    model = tf.keras.Sequential()
    model.add(InputLayer(input_shape = input_shape))
    if batch_normalization: 
        model.add(BatchNormalization(momentum=momentum))
    model.add(Conv2D(32, 3, strides = 1, padding = 'same'))
    model.add(Activation(activation = activation))
    if batch_normalization: 
        model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(64, 3, strides = 1, padding = 'same'))
    model.add(Activation(activation = activation))
    if batch_normalization: 
        model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024, activation = activation, kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.01)))
    model.add(Dropout(0.5))
    if loss == 'binary_crossentropy':
        model.add(Dense(1, activation = 'sigmoid'))
    elif loss == 'sparse_categorical_crossentropy':
        model.add(Dense(2, activation = 'softmax'))
    
    if optimizer == 'adam':
      model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.9999, epsilon=1e-07), metrics=['accuracy'])
    elif optimizer == 'sgd':
      model.compile(loss=loss, optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.0), metrics=['accuracy'])     
    
    return model

In [ ]:
#Callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=10, min_lr=0.0001, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=100, min_delta=0.01, restore_best_weights=True)

In [ ]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X, y):

  # Define the model architecture
  model = build_model(input_shape = (7, 7, 20), batch_normalization=True, momentum = 0.99, activation = 'relu', loss='binary_crossentropy', lr = 0.0001, optimizer='adam')

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(X[train], y[train],
              batch_size=32,
              epochs=200,
              callbacks=[reduce_lr, early_stop],
              validation_data = (X[test], y[test]), verbose=0)
  
  # Generate generalization metrics
  scores = model.evaluate(X[test], y[test], verbose=0)
  print(f'Score for fold {fold_no} on validation: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = model.evaluate(X[train], y[train], verbose=0)
  print(f'Score for fold {fold_no} on training: {model.metrics_names[0]} of {scores_train[0]}; {model.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train.append(scores_train[1] * 100)
  loss_per_fold_train.append(scores_train[0])

  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train[i]} - Accuracy: {acc_per_fold_train[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train)} (+- {np.std(acc_per_fold_train)})')
print(f'> Loss: {np.mean(loss_per_fold_train)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Restoring model weights from the end of the best epoch.
Epoch 00112: early stopping
Score for fold 1 on validation: loss of 0.6529799699783325; accuracy of 64.26066160202026%
Score for fold 1 on training: loss of 0.6234534978866577; accuracy of 67.20819473266602%
------------------------------------------------------------------------
Training for fold 2 ...
Restoring model weights from the end of the best epoch.
Epoch 00104: early stopping
Score for fold 2 on validation: loss of 0.7481210827827454; accuracy of 63.79914879798889%
Score for fold 2 on training: loss of 0.7388977408409119; accuracy of 64.36977982521057%
------------------------------------------------------------------------
Training for fold 3 ...
Restoring model weights from the end of the best epoch.
Epoch 00103: early stopping
Score for fold 3 on validation: loss of 0.9030749797821045; accuracy of 63.060736656188965%
Score